In [1]:
import json
import time
import pandas
import requests
from bs4 import BeautifulSoup

In [3]:
!pip install trafilatura
import os
import re
import shutil
import trafilatura

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 21.8 MB/s eta 0:00:00


In [2]:
url = "https://www.stonerealestate.com.au/buy/commercial/"
res = requests.get(url)
soup = BeautifulSoup(res.text, "html.parser")
soup

<!DOCTYPE html>

<html dir="ltr" itemscope="" itemtype="https://schema.org/WebPage" lang="en-US" xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml">
<head profile="https://gmpg.org/xfn/11">
<meta content="width=device-width, minimum-scale=1.0, maximum-scale=1.0, user-scalable=yes" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
<script type="text/javascript">
/* <![CDATA[ */
 var gform;gform||(document.addEventListener("gform_main_scripts_loaded",function(){gform.scriptsLoaded=!0}),window.addEventListener("DOMContentLoaded",function(){gform.domLoaded=!0}),gform={domLoaded:!1,scriptsLoaded:!1,initializeOnLoaded:function(o){gform.domLoaded&&gform.scriptsLoaded?o():!gform.domLoaded&&gform.scriptsLoaded?window.addEventListener("DOMContentLoaded",o):document.addEventListener("gform_main_scripts_loaded",o)},hooks:{action:{},filter:{}},addAction:function(o,n,r,t){gform.addHook("action",o,n,r,t)},addFilter:function(o,n,r,t)

In [10]:
from urllib.parse import urljoin, urlparse

def get_stone_realestate_urls(soup, base_url="https://www.stonerealestate.com.au"):
    urls = set()

    for a in soup.find_all("a", href=True):
        href = a["href"].strip()
        urls.add(href)
        # print(href)

        # # Convert relative URLs to absolute
        # full_url = urljoin(base_url, href)

        # parsed = urlparse(full_url)

        # # Check domain
        # if parsed.netloc == "https://www.stonerealestate.com.au":
        #     urls.add(full_url)

    return sorted(urls)

urls = get_stone_realestate_urls(soup)

for url in urls:
    print(url)

#
http://thestoneshop.com.au/
https://stone.lightspeedvt.com/
https://stone.supplycrew.com.au/
https://www.agentbox.com.au/features-pages/websites
https://www.facebook.com/pages/Stone-Real-Estate/123695347694910
https://www.instagram.com/STONE_Real_Estate/
https://www.linkedin.com/company/stone-real-estate-?trk=tyah&trkInfo=clickedVertical%3Acompany%2CclickedEntityId%3A6405155%2Cidx%3A3-2-7%2CtarId%3A1450153664292%2Ctas%3AStone+real+estate
https://www.lookatmystrata.com.au/login/966851
https://www.stonerealestate.com.au
https://www.stonerealestate.com.au/about/
https://www.stonerealestate.com.au/acreage/
https://www.stonerealestate.com.au/blog/
https://www.stonerealestate.com.au/buy/acreage/
https://www.stonerealestate.com.au/buy/commercial/
https://www.stonerealestate.com.au/buy/off-market-sales/
https://www.stonerealestate.com.au/buy/property-alerts/
https://www.stonerealestate.com.au/buy/residential/
https://www.stonerealestate.com.au/careers/
https://www.stonerealestate.com.au/fran

In [ ]:
url = "https://www.stonerealestate.com.au/wp-content/plugins/zoorealty/display/elements/load_listings.php?post_id=116&status=1,4&list=ResidentialSale&order=&order_direction=&property_type=&keywords=&price_min=&price_max=&bedrooms=&bathrooms=&carspaces=&sur_suburbs=&related_office=&office_id=&exclude_office_id=&page=17"
res = requests.get(url)
res.text

In [12]:
from bs4 import BeautifulSoup

def parse_property_listings(soup):
    listings = []

    for card in soup.select("div.column.listing"):
        data = {}

        # --- URL & Image ---
        link = card.select_one(".image a.centerimage")
        data["url"] = link["href"] if link else None

        img = card.select_one(".image img")
        data["image_url"] = img["src"] if img else None

        # --- Status (Under offer etc.) ---
        sticker = card.select_one(".sticker")
        data["status"] = sticker.get_text(strip=True) if sticker else None

        # --- Location ---
        data["suburb"] = (
            card.select_one(".title .suburb a").get_text(strip=True)
            if card.select_one(".title .suburb a")
            else None
        )

        data["address"] = (
            card.select_one(".title .address").get_text(strip=True)
            if card.select_one(".title .address")
            else None
        )

        # --- Rooms ---
        rooms = card.select(".rooms .item")
        data["bedrooms"] = None
        data["bathrooms"] = None
        data["carspaces"] = None
        data["has_study"] = False

        for r in rooms:
            text = r.get_text(strip=True)
            icon = r.select_one("i")

            if not icon:
                continue

            cls = " ".join(icon.get("class", []))

            if "icon-bedrooms" in cls:
                data["bedrooms"] = int(text.split()[0])
            elif "icon-bathrooms" in cls:
                data["bathrooms"] = int(text.split()[0])
            elif "icon-carspaces" in cls:
                data["carspaces"] = int(text.split()[0])
            elif "icon-study" in cls:
                data["has_study"] = True

        # --- Price & Open Times ---
        price_block = card.select_one(".price")

        data["price"] = None
        data["open_times"] = None

        if price_block:
            default_price = price_block.select_one(".default-price")
            if default_price:
                data["price"] = default_price.get_text(strip=True)

            open_amount = price_block.select_one(".opens-price .amount")
            if open_amount:
                open_text = open_amount.get_text(strip=True)
                if open_text:
                    data["open_times"] = open_text

        listings.append(data)

    return listings

url = "https://www.stonerealestate.com.au/wp-content/plugins/zoorealty/display/elements/load_listings.php?post_id=116&status=1,4&list=ResidentialSale&order=&order_direction=&property_type=&keywords=&price_min=&price_max=&bedrooms=&bathrooms=&carspaces=&sur_suburbs=&related_office=&office_id=&exclude_office_id=&page=17"
res = requests.get(url)
soup = BeautifulSoup(res.text, "html.parser")
properties = parse_property_listings(soup)

for p in properties:
    print(p)

{'url': 'https://www.stonerealestate.com.au/property/8566886-16-1684-channel-highway-margate-tas/', 'image_url': 'https://phimg.reapit.website/bfdc4d00f9a79f51d5c09c0fc195d2ea693676d8', 'status': 'Under offer', 'suburb': 'Margate', 'address': '16/1684 Channel Highway', 'bedrooms': 2, 'bathrooms': 1, 'carspaces': 2, 'has_study': False, 'price': 'Offers over $525,000', 'open_times': None}
{'url': 'https://www.stonerealestate.com.au/property/8566857-10-1-3-wentworth-street-manly-nsw/', 'image_url': 'https://phimg.reapit.website/6c6a00f77695dc14f3891d4d005bad87ad8a2159', 'status': None, 'suburb': 'Manly', 'address': '10/1-3 Wentworth Street', 'bedrooms': 2, 'bathrooms': 2, 'carspaces': 2, 'has_study': False, 'price': 'Auction', 'open_times': 'Sat Jan 10 9:30-10:00am'}
{'url': 'https://www.stonerealestate.com.au/property/8566856-4-8-james-street-manly-nsw/', 'image_url': 'https://phimg.reapit.website/6732b07574cfee6e6b9cf7d462bc1f9c3b7f5e19', 'status': None, 'suburb': 'Manly', 'address': '4

In [17]:
import requests
from bs4 import BeautifulSoup

BASE_URL = (
    "https://www.stonerealestate.com.au/wp-content/plugins/zoorealty/"
    "display/elements/load_listings.php"
)

PARAMS = {
    "post_id": "119",
    "status": "1,4",
    "list": "commercial_sale",
    "order": "",
    "order_direction": "",
    "property_type": "",
    "keywords": "",
    "price_min": "",
    "price_max": "",
    "bedrooms": "",
    "bathrooms": "",
    "carspaces": "",
    "sur_suburbs": "",
    "related_office": "",
    "office_id": "",
    "exclude_office_id": "",
}

all_properties = []
page = 1

while True:
    PARAMS["page"] = page

    print(f"Fetching page {page}...")

    res = requests.get(BASE_URL, params=PARAMS, timeout=15)
    res.raise_for_status()

    soup = BeautifulSoup(res.text, "html.parser")
    properties = parse_property_listings(soup)

    # Stop if no properties returned
    if not properties:
        print("No properties found, stopping.")
        break

    all_properties.extend(properties)

    print(f"→ Found {len(properties)} properties")

    # Stop condition: last page has fewer than 12 items
    if len(properties) < 12:
        print("Last page reached.")
        break

    page += 1

print(f"\nTotal properties collected: {len(all_properties)}")


Fetching page 1...
→ Found 12 properties
Fetching page 2...
→ Found 12 properties
Fetching page 3...
→ Found 11 properties
Last page reached.

Total properties collected: 35


In [18]:
import json

output_filename = 'commercial_sale.json'

with open(output_filename, 'w') as f:
    json.dump(all_properties, f, indent=4)

print(f"Successfully saved {len(all_properties)} properties to {output_filename}")

Successfully saved 35 properties to commercial_sale.json
